## Detector de anomalias en tarjetas de credito.

### Contexto:
Es importante que las compañías de tarjetas de crédito puedan reconocer transacciones fraudulentas de tarjetas de crédito para que a los clientes no se les cobre por los artículos que no compraron.

### Contenido:
Los conjuntos de datos contienen transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos. Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está altamente desequilibrado, la clase positiva (fraudes) representa el 0.172% de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA (Principal Component Analysis). Desafortunadamente, debido a problemas de confidencialidad, no podemos proporcionar las características originales y más información de fondo sobre los datos. Las características V1, V2, ... V28 son los componentes principales obtenidos con PCA, las únicas características que no se han transformado con PCA son 'Tiempo' y 'Cantidad'. La función 'Tiempo' contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La característica 'Cantidad' es la Cantidad de la transacción, esta característica se puede utilizar para el aprendizaje sensible al costo dependiente del ejemplo. La característica 'Clase' es la variable de respuesta y toma el valor 1 en caso de fraude y 0 en caso contrario.

### Proposito:
Identificar transacciones fraudulentas en tarjetas de credito

Dada la relación de desequilibrio de clase, recomendamos medir la precisión utilizando el área bajo la curva de precisión-recuperación (AUPRC). La precisión de la matriz de confusión no es significativa para la clasificación desequilibrada.

### Agradecimientos y creditos:
El conjunto de datos se ha recopilado y analizado durante una colaboración de investigación de Worldline y el Machine Learning Group (http://mlg.ulb.ac.be) de ULB (Université Libre de Bruxelles) sobre minería de datos grandes y detección de fraude. Más detalles sobre proyectos actuales y pasados sobre temas relacionados están disponibles en https://www.researchgate.net/project/Fraud-detection-5 y la página del proyecto DefeatFraud y a Krish C Naik creador del repositorio de donde fue tomado este modelo disponible en GitHub https://github.com/krishnaik06/Credit-Card-Fraudlent usado bajo la licencia GNU General Public License v3.0.

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]


In [ ]:
data = pd.read_csv('creditcard.csv',sep=',')
data.head()

In [ ]:
data.info()

## Exploratory Data Analysis

In [ ]:
data.isnull().values.any()

In [ ]:
count_classes = pd.value_counts(data['Class'], sort = True)

count_classes.plot(kind = 'bar', rot=0)

plt.title("Distribución de clase de transacción")

plt.xticks(range(2), LABELS)

plt.xlabel("Class")

plt.ylabel("Frequency")

In [ ]:
## Obtenga el Fraude y el conjunto de datos normal 

fraud = data[data['Class']==1]

normal = data[data['Class']==0]

In [ ]:
print(fraud.shape,normal.shape)

In [ ]:
## Necesitamos analizar más cantidad de información de los datos de la transacción.
#¿Qué tan diferente es la cantidad de dinero utilizada en diferentes clases de transacciones? 
fraud.Amount.describe()

In [ ]:
normal.Amount.describe()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Amount per transaction by class')
bins = 50
ax1.hist(fraud.Amount, bins = bins)
ax1.set_title('Fraud')
ax2.hist(normal.Amount, bins = bins)
ax2.set_title('Normal')
plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.xlim((0, 20000))
plt.yscale('log')
plt.show();

In [ ]:
# Verificaremos ¿Las transacciones fraudulentas ocurren con mayor frecuencia durante cierto período de tiempo? Descubrámoslo con una representación visual.

f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Tiempo de transaccion vs Cantidad por clase')
ax1.scatter(Fraud.Time, Fraud.Amount)
ax1.set_title('Fraude')
ax2.scatter(Normal.Time, Normal.Amount)
ax2.set_title('Normal')
plt.xlabel('Tiempo (en segundos)')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
## Take some sample of the data

data1= data.sample(frac = 0.1,random_state=1)

data1.shape

In [ ]:
data.shape

In [ ]:
#Determine el número de fraudes y transacciones válidas en el conjunto de datos 

Fraud = data1[data1['Class']==1]

Valid = data1[data1['Class']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

In [ ]:
print(outlier_fraction)

print("Casos de fraude: {}".format(len(Fraud)))

print("Casos validos: {}".format(len(Valid)))

In [ ]:
## Correlation
import seaborn as sns
#get correlations of each features in dataset
corrmat = data1.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
#Crear características independientes y dependientes 
columns = data1.columns.tolist()
# Filtre las columnas para eliminar datos que no queremos 
columns = [c for c in columns if c not in ["Class"]]
# Almacene la variable que estamos prediciendo 
target = "Class"
# Definir un estado aleatorio. 
state = np.random.RandomState(42)
X = data1[columns]
Y = data1[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

## Predicción del modelo





Ahora es el momento de comenzar a construir el modelo. Los tipos de algoritmos que vamos a utilizar para tratar de detectar anomalías en este conjunto de datos son los siguientes

### Aislamiento del Algoritmo Forestal :
Una de las técnicas más nuevas para detectar anomalías se llama bosques de aislamiento. El algoritmo se basa en el hecho de que las anomalías son puntos de datos que son pocos y diferentes. Como resultado de estas propiedades, las anomalías son susceptibles a un mecanismo llamado aislamiento.

Este método es muy útil y es fundamentalmente diferente de todos los métodos existentes. Introduce el uso del aislamiento como un medio más efectivo y eficiente para detectar anomalías que las medidas básicas de distancia y densidad comúnmente utilizadas. Además, este método es un algoritmo con una baja complejidad de tiempo lineal y un pequeño requisito de memoria. Construye un modelo de buen rendimiento con una pequeña cantidad de árboles que utilizan pequeñas submuestras de tamaño fijo, independientemente del tamaño de un conjunto de datos. 

Los métodos típicos de aprendizaje automático tienden a funcionar mejor cuando los patrones que intentan aprender son equilibrados, lo que significa que la misma cantidad de comportamientos buenos y malos están presentes en el conjunto de datos.

Como funciona el bosque de aislamiento?
El algoritmo de bosque de aislamiento aísla las observaciones seleccionando aleatoriamente una característica y luego seleccionando aleatoriamente un valor dividido entre los valores máximo y mínimo de la característica seleccionada. El argumento lógico dice: aislar las observaciones de anomalías es más fácil porque solo se necesitan unas pocas condiciones para separar esos casos de las observaciones normales. Por otro lado, aislar observaciones normales requiere más condiciones. Por lo tanto, se puede calcular un puntaje de anomalía como el número de condiciones requeridas para separar una observación dada.

La forma en que el algoritmo construye la separación es primero creando árboles de aislamiento o árboles de decisión aleatorios. Luego, la puntuación se calcula como la longitud de la ruta para aislar la observación.


### Algoritmo de factor de valor atípico local (LOF)
El algoritmo LOF es un método de detección de valores atípicos no supervisados que calcula la desviación de densidad local de un punto de datos dado con respecto a sus vecinos. Considera como muestras atípicas que tienen una densidad sustancialmente menor que sus vecinas.

El número de vecinos considerado (parámetro n_neighbours) generalmente se elige 1) mayor que el número mínimo de objetos que debe contener un clúster, para que otros objetos puedan ser valores atípicos locales en relación con este clúster y 2) más pequeño que el número máximo de cerca de objetos que potencialmente pueden ser valores atípicos locales. En la práctica, tales informaciones generalmente no están disponibles, y tomar n_neighbours = 20 parece funcionar bien en general..

In [ ]:
##Definir los métodos de detección de valores atípicos.

classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier_fraction,random_state=state, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier_fraction),
    "Support Vector Machine":OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, 
                                         max_iter=-1, random_state=state)
   
}

In [ ]:
type(classifiers)

In [ ]:
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Ajustar los datos y etiquetar valores atípicos
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
    elif clf_name == "Support Vector Machine":
        clf.fit(X)
        y_pred = clf.predict(X)
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
#Cambie la forma de los valores de predicción a 0 para transacciones válidas, 1 para transacciones de fraude    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != Y).sum()
# Ejecutar métricas de clasificación
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(Y,y_pred))
    print("Classification Report :")
    print(classification_report(Y,y_pred))

#### Observaciones:
- El bosque de aislamiento detectó 73 errores frente al factor de valor atípico local que detectó 97 errores frente a SVM que detectó 8516 errores
- El bosque de aislamiento tiene un 99.74% más de precisión que un LOF de 99.65% y un SVM de 70.09
- Al comparar la precisión y recuperación de errores para 3 modelos, el Bosque de aislamiento funcionó mucho mejor que el LOF, ya que podemos ver que la detección de casos de fraude es de alrededor del 27% frente a la tasa de detección de LOF de solo el 2% y SVM del 0%.
- Por lo tanto, el método general de bosque de aislamiento funcionó mucho mejor para determinar los casos de fraude, que es de alrededor del 30%.
- También podemos mejorar esta precisión al aumentar el tamaño de la muestra o usar algoritmos de aprendizaje profundo, sin embargo, a costa del gasto computacional. También podemos usar modelos de detección de anomalías complejas para obtener una mayor precisión en la determinación de casos más fraudulentos